In [ ]:
import json
from collections import defaultdict
from copy import deepcopy
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Rectangle

import scipy.stats as stats
from scipy.stats import norm, kendalltau
from statsmodels.stats.multitest import multipletests
from tqdm.auto import tqdm

from utils import read_data_into_dataframe, qualities
sns.set(color_codes=True)

In [ ]:
all_data = read_data_into_dataframe()

In [ ]:
auto_measures = [
    "bertscores_F",
    "rougeL",
    "rouge1",
    "rouge2",
    "bleu",
    "jshannon",
    "blanc",
    "estime",
]

renamer = {
    'bertscores_F': 'BERTScore',
    'rougeL': "ROUGE-L",
    'rouge1': "ROUGE-1",
    'rouge2': "ROUGE-2",
    'bleu': "BLEU",
    'jshannon':"JS",
    'blanc':"BLANC",
    'estime': 'ESTIME'
}
figure1_columns = ["Metric", "EN-DE", "EN-FR", "EN-ES", "EN-IT", "EN-AF", "EN-HI", "EN-RU"]

negative_corrs = ["jshannon", "estime"]
non_en = all_data.loc[all_data.language != "en", "language"].drop_duplicates().tolist()

In [ ]:
def calc_true_correlations(all_scores):
    # transform [experts, en, es, fr, ...] -> [experts, language, value]
    long_data = pd.melt(all_scores, id_vars="experts", var_name="language", value_name="value")

    # calc correlations between experts and each language
    corrs = long_data.groupby("language").apply(
        lambda x: kendalltau(x.experts, x['value'], "c").correlation
    )
    
    # calc correlation differences between english and non eng languages
    # return df of form [language, difference]
    return (
        corrs
        .reset_index(name='true')
        .assign(single_index=1)
        .pivot(index="single_index", columns="language")["true"]
        .melt(id_vars=["en"])
        .assign(true_difference=lambda x: x.en - x.value)
        .loc[:,['language', 'true_difference']]
    )


def run_bootstrap(all_scores, n_samples=1000):
    
    # create df of sampled indexes for each bootstrap sample
    bootidxs = {idx: np.random.randint(1700, size=1700) for idx in range(n_samples)}
    idx_frame = pd.melt(pd.DataFrame(bootidxs), var_name='sample_no', value_name='idx')
    
    # create bootstrap samples by merging with data
    boot_samples = idx_frame.merge(all_scores, left_on="idx", right_index=True).sort_values(
        ["sample_no", "idx"]
    )
    
    # pivot from wide [sample, idx, experts, en, fr, ...] 
    # to long [sample, idx, experts, language, value]
    boot_tidy = pd.melt(
        boot_samples,
        id_vars=["sample_no", "idx", "experts"],
        value_name="value",
        var_name="language",
    )
    
    # calc correlation differences btwn en and other langs using each bootstrap sample
    correlations = (
        boot_tidy
        .groupby(["sample_no", "language"])
        .apply(lambda x: kendalltau(x["experts"], x["value"], "c").correlation)
        .reset_index(name="correlation")
        .pivot(index='sample_no', columns='language')['correlation']
        .melt(id_vars=['en'])
        .assign(difference = lambda x: x['en'] - x['value'])
    )
    
    return correlations


def bootstrap_correlation_scores(df, quality_names, metrics, n_samples = 10000, seed=123):
    np.random.seed(seed)
    corr_dfs = []
    for qual in tqdm(quality_names):
        for met in tqdm(metrics):

            expert = df.loc[
                (df.submetric == qual)
                & (df.metric == "experts")
                & (df.language == "en"),
                "value",
            ].reset_index(drop=True)

            english = df.loc[
                (df.submetric == met) & (df.language == "en"), "value"
            ].reset_index(drop=True)

            df_list = [expert, english]

            for lang in non_en:
                df_list.append(df.loc[
                    (df.submetric == met) & (df.language == lang), "value"
                ].reset_index(drop=True))


            all_scores = pd.concat(df_list, axis=1)
            all_scores.columns = ["experts", "en"] + non_en

            correlations = run_bootstrap(all_scores, n_samples=n_samples)
            true_corrs = calc_true_correlations(all_scores)

            correlations['metric'] = met
            correlations['quality'] = qual
            corr_dfs.append(correlations.merge(true_corrs, on='language'))

    corr = pd.concat(corr_dfs)
    return corr


def tost_test(corr, margin=0.1):
    corr = deepcopy(corr)
    
    if isinstance(margin, float):    
        corr['higher_hypothesis'] = (corr['difference'] > margin)
        corr['lower_hypothesis'] = (corr['difference'] < -margin)
    else:
        corr['higher_hypothesis'] = (corr['difference'] > corr[f'margin_{margin}'])
        corr['lower_hypothesis'] = (corr['difference'] < -corr[f'margin_{margin}'])
    
    tost = (
        corr.groupby(["metric", "language", "quality"])[
            ["higher_hypothesis", "lower_hypothesis"]
        ]
        .mean()
        .reset_index()
    )
    tost["pvalue"] = tost.apply(
        lambda x: max(x["higher_hypothesis"], x["lower_hypothesis"]), axis=1
    )

    return tost


def tost_corrected(tost_df, margin_name):
    reject, corrected, _, _ = multipletests(
        tost_df.pvalue,
        alpha=0.05,
        method="fdr_by",
        is_sorted=False,
        returnsorted=False,
    )
    print(
        f"Pre Correction Rejection % {tost_df.pvalue.apply(lambda x: x <= 0.05).mean()*100:.2f}"
    )
    print(f"BY Correction Rejection % {reject.mean()*100:.2f}")
    tost_df["reject_by"] = reject
    tost_df["pval_by"] = corrected

    plt.figure(figsize=(5, 3))
    tost_df.pvalue.hist(alpha=0.3, label="Before Correction")
    tost_df.pval_by.hist(alpha=0.3, label=" B-Y Corrected")

    plt.xticks(size=14)
    plt.yticks(size=14)
    plt.xlabel("p-value", size=14)
    plt.ylabel("Count", size=14)
    plt.title(f"{margin_name} Margin", size=15)
    plt.legend()
    plt.show()

Warning - bootstrap may take some time to run. Set `new_run=True` to run.

## Run Bootstrap

In [ ]:
new_run = False
n_samples = 10000
if new_run:
    corr = run_bootstrap(all_data, n_samples)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    corr.to_csv(f"bootstrap_forwardtranslation_{n_samples}iters_{timestamp}", index=False)

In [ ]:
with open("data/margins.json") as f:
    margins = json.load(f)

corr['margin_std'] = corr.apply(lambda x: margins['std'][x.metric][x.quality], axis=1)
corr['margin_maxdiff'] = corr.apply(lambda x: margins['max_margin'][x.metric][x.quality], axis=1)

## Calculate TOST results for Difference Margins

In [ ]:
tost_maxdiff = tost_test(corr, margin='maxdiff')
tost_corrected(tost_maxdiff, 'Max Difference')

In [ ]:
tost_std = tost_test(corr, margin='std')
tost_corrected(tost_std, 'Standard Deviation')

In [ ]:
tost_constant = tost_test(corr, margin=0.05)
tost_corrected(tost_constant, 'Constant')

## Figure 3

In [ ]:
trace = []

for margin in np.linspace(0.001, 0.25, 100):
    tost = tost_test(corr, margin=margin)

    trace.append(
        tost
        .groupby(['quality'])
        ['pvalue']
        .agg([('total', 'count'), ('reject', lambda x: (x<=0.05).sum())])
        .reset_index()
        .assign(margin=margin)
        .assign(pct=lambda x: 1.0 * x["reject"] / x['total'])
    )

In [ ]:
remap = defaultdict(list)
for mtype, sub in margins.items():
    for metric, qualdict in sub.items():
        if metric in auto_measures:
            for qual, val in qualdict.items():
                remap['margin'].append(mtype)
                remap['metric'].append(metric)
                remap['quality'].append(qual)
                remap['value'].append(val)
                
margin_df = pd.DataFrame(remap)

In [ ]:
margin_df

In [ ]:
for quality in qualities:
    temp = margin_df.loc[(margin_df.margin == 'std') & (margin_df.quality == quality)]
    print(quality)
    print(f"Median: {np.median(temp.value)}")
    print(f"IQR: {stats.iqr(temp.value)}")

          

In [ ]:
def sig_test_grid(df, quality, ax):
    
    metrics = df.metric.drop_duplicates()
    data = np.ones((len(metrics), len(non_en)))
    highlights = []
    
    for i, metric in enumerate(metrics):
        for j, language in enumerate(non_en):
          
            value = df.loc[
                (df.metric == metric)
                & (df.language == language)
                & (df.quality == quality),
                "pvalue",
            ].item()
            reject_by = df.loc[
                (df.metric == metric)
                & (df.language == language)
                & (df.quality == quality),
                "reject_by",
            ].item()
            data[i, j] = value
           

            if reject_by:
                highlights.append((i, j))

  
    colors = ["#2061A9", "#5C9FCD", "#C7DBF0", "#FFFFFF"]
    boundaries = [0, 0.05, 1.0]
    norm = matplotlib.colors.BoundaryNorm(boundaries=boundaries, ncolors=256)
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
    sns.heatmap(
        data,
        ax=ax,
        cbar=False,
        xticklabels=list(map(str.upper, non_en)),
        yticklabels=[renamer.get(x) for x in metrics],
        linewidths=0.5,
        linecolor="lightgray",
        cmap=cmap,
        norm=norm,
        cbar_ax=None,
        label=quality
    )

    ax.set_title(quality.title(), size=16)
    ax.tick_params(axis='both', which='major', labelsize=16)

    for i, j in highlights:
        ax.add_patch(Rectangle((j, i), 1, 1, fill=True, alpha =1, edgecolor="red", lw=3))
        
        

In [ ]:
fig, axes = plt.subplots(2,2, sharex=True, sharey=True, figsize=(8, 8))
for i, quality in enumerate(tost_std.quality.drop_duplicates()):
    a = i % 2
    b = 0 if i < 2 else 1
    
    sig_test_grid(tost_std, quality, axes[a,b])
   
    
plt.savefig('tost_std_box.pdf', bbox_inches='tight', format='pdf')


In [ ]:
fig, axes = plt.subplots(2,2, sharex=True, sharey=True, figsize=(8,8))
for i, quality in enumerate(tost_std.quality.drop_duplicates()):
    a = i % 2
    b = 0 if i < 2 else 1
    
    sig_test_grid(tost_constant, quality, axes[a,b])

plt.savefig('tost_constant_box.pdf', bbox_inches='tight', format='pdf')

## Figure 4

In [ ]:
sns.set()
margin_df.loc[(margin_df.margin == 'std'), ['quality', 'value']].boxplot(
    by='quality',
    backend='matplotlib',
    figsize=(7,4.5),
    boxprops= dict(linewidth=2.2, color='black'), 
    whiskerprops=dict(linestyle='-',linewidth=2.2, color='black'),
    capprops=dict(linestyle='-',linewidth=2.2, color='black'),
    medianprops=dict(linestyle='-',linewidth=2.2, color='gray')
    
)
plt.title('Distribution of Margins by Quality', size=17)
plt.xlabel("")
plt.ylabel("Margin of Equivalence", size=17)
plt.suptitle('')
plt.xticks(size=17)
plt.yticks(size=17)
plt.savefig("boxplot.pdf", bbox_inches="tight", format='pdf')

In [ ]:
pd.concat(trace).pivot(index="margin", columns="quality")["pct"].plot(figsize=(7, 4.5))
plt.xlabel("Margin of Equivalence", size=17)
plt.ylabel("% of P-Values Under 0.05", size=17)
plt.title("TOST Sensitivity to Margin of Equivalence", size=17)
plt.xticks(size=17)
plt.yticks(size=17)
plt.legend(fontsize=17)

plt.savefig("tost_sensitivity.pdf", bbox_inches="tight",format='pdf')

## Anderson Hauck

In [ ]:
# exp = 1
# en = 2
# other language = 3
# r12 = exp - en correlation
# r13 = exp - other correlation
# r23 = en - other correlation

def anderson_hauck(r12, r13, r23, N, delta=0.1):
    """
    See https://yorkspace.library.yorku.ca/xmlui/bitstream/handle/10315/34580/Counsell_Cribbie.pdf
    Page 297
    """
    R = (1 - r12**2 - r13**2 - r23**2) + (2 * r12 * r13 * r23)
    stat1 = (
        (abs(r12-r13) - delta) * 
        np.sqrt(
            ((N-1)*(1+r23)) / 
            (2*((N-1)/(N-3)) * R + 0.25*(r12+r13)**2 * (1 - r23)**3)
        )
    )

    stat2 = (
        (-1*abs(r12-r13) - delta) * 
        np.sqrt(
            ((N-1)*(1+r23)) / 
            (2*((N-1)/(N-3)) * R + 0.25*(r12+r13)**2 * (1 - r23)**3)
        )
    )
    
    p = norm.pdf(stat1) - norm.pdf(stat2)
    
    return p


def ah_corrected(df, margin_name):
    reject, corrected, _, _ = multipletests(
        df.pvalue,
        alpha=0.05,
        method="fdr_by",
        is_sorted=False,
        returnsorted=False,
    )
    print(f"Pre Correction Rejection % {df.pvalue.apply(lambda x: x <= 0.05).mean()*100:.2f}")
    print(f"BY Correction Rejection % {reject.mean()*100:.2f}")
    df["reject_by"] = reject
    df["pval_by"] = corrected
    df['metric'] = df.submetric
    df['language'] = df.language.apply(str.lower)
    
    plt.figure(figsize=(5,3))
    df.pvalue.hist(alpha=0.3, label='Before Correction', bins=10)
    df.pval_by.hist(alpha=0.3, label=' B-Y Corrected',bins=10)
    
    plt.xticks(size=14)
    plt.yticks(size=14)
    plt.xlabel("p-value", size=14)
    plt.ylabel("Count", size=14)
    plt.title(f"{margin_name} Margin", size=15)
    plt.legend()
    plt.show()
    

def run_anderson_hauck(df, margin):
    N = 1700
    ah_results = []

    for idx, row in df.iterrows():
        r12 = row['en_exp_corr']
        r13 = row['lang_exp_corr']
        r23 = row['lang_en_corr']
        delta = row[margin]
        row['pvalue'] = anderson_hauck(r12, r13, r23, N, delta=delta)

        ah_results.append(dict(row))
        
    return pd.DataFrame(ah_results)
    

In [ ]:
ah_dict = []
for met in auto_measures:
    for qual in qualities:
        for lang in all_data.language.drop_duplicates().tolist():

            expert = all_data.loc[
                (all_data.submetric == qual)
                & (all_data.metric == "experts")
                & (all_data.language == "en"),
                "value",
            ]
            
            english = all_data.loc[
                (all_data.submetric == met)
                & (all_data.language == "en"),
                "value",
            ]
            
            other_met = all_data.loc[(all_data.submetric == met) & (all_data.language == lang), "value"]

            lang_exp_corr = kendalltau(expert, other_met).correlation
            lang_en_corr = kendalltau(english, other_met).correlation

            ah_dict.append(
                dict(
                    submetric=met,
                    quality=qual.title(),
                    language=lang.upper(),
                    lang_exp_corr=lang_exp_corr,
                    lang_en_corr=lang_en_corr
                )
            )

ah_df = pd.DataFrame(ah_dict).sort_values(["quality", "submetric", "language"]).reset_index(drop=True)
ah_df.loc[ah_df.submetric.isin(negative_corrs), ["lang_exp_corr"]] = (
    ah_df.loc[ah_df.submetric.isin(negative_corrs), ["lang_exp_corr"]] * -1
)

english_corrs = (
    ah_df
    .loc[ah_df.language == 'EN']
    .drop(['lang_en_corr', 'language'], axis=1)
)

ah_test = (
    ah_df.merge(english_corrs, on=['submetric', 'quality'], suffixes=["", "_en"])
    .rename(columns={'lang_exp_corr_en':'en_exp_corr'})
    .loc[ah_df.language != 'EN']
)

ah_test['margin_std'] = ah_test.apply(lambda x: margins['std'][x.submetric][x.quality.lower()], axis=1)
ah_test['margin_maxdiff'] = ah_test.apply(lambda x: margins['max_margin'][x.submetric][x.quality.lower()], axis=1)
ah_test['margin_constant'] = 0.05

In [ ]:
ah_std = run_anderson_hauck(ah_test, 'margin_std')
ah_corrected(ah_std, "Standard Deviation")

In [ ]:
ah_maxdiff = run_anderson_hauck(ah_test, 'margin_maxdiff')
ah_corrected(ah_maxdiff, "Max Diff")

In [ ]:
ah_constant = run_anderson_hauck(ah_test, 'margin_constant')
ah_corrected(ah_constant, "Constant")

In [ ]:
fig, axes = plt.subplots(2,2, sharex=True, sharey=True, figsize=(8,8))
for i, quality in enumerate(ah_std.quality.drop_duplicates()):
    a = i % 2
    b = 0 if i < 2 else 1
    
    sig_test_grid(ah_std, quality, axes[a,b])

plt.show()